In [2]:
import torch
import transformers
from huggingface_hub import snapshot_download
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import cv2
import requests
from io import BytesIO
from pathlib import Path
import sys
import warnings
warnings.filterwarnings("ignore")
transformers.logging.set_verbosity_error()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"torch version: {torch.__version__}, using device: {device}")
print(f"transformers version: {transformers.__version__}")

torch version: 2.8.0+cu126, using device: cuda
transformers version: 4.57.1


In [ ]:
if "google.colab" in sys.modules: #Detectamos si estamos en google colab o en local
    from google.colab import drive
    drive.mount('/content/drive')
    HF_MODELS_DIR = Path("/content/drive/MyDrive/hf_models")
else:
    HF_MODELS_DIR = Path(r"G:\Mi unidad\hf_models")

HF_MODELS_DIR.mkdir(parents=True, exist_ok=True)

models = [
    # "IDEA-Research/grounding-dino-base",
    # "facebook/sam-vit-large",
    # "openai/clip-vit-base-patch32",
    # "Salesforce/blip-image-captioning-base",
    "facebook/detr-resnet-50"
]

for repo in models: #Atención! descargará unos 7 GB a Google Drive
    local_dir = HF_MODELS_DIR / repo
    snapshot_download(
        repo_id=repo,
        local_dir=str(local_dir),
        # local_dir_use_symlinks=False,
        resume_download=True
)

Fetching 6 files:   0%|          | 0/6 [00:00<?, ?it/s]

LookupError: <ContextVar name='shell_parent' at 0x00000235412EF880>

In [ ]:
from transformers import pipeline
from PIL import Image, ImageDraw, ImageFont
import requests
import matplotlib.pyplot as plt

# Crear el pipeline de detección
detector = pipeline("object-detection", model="facebook/detr-resnet-50")

# Cargar una imagen de ejemplo (puedes cambiar la URL o usar un archivo local)
url = "https://huggingface.co/datasets/Narsil/image_dummy/raw/main/parrots.png"
image = Image.open(requests.get(url, stream=True).raw)

# Realizar la detección
results = detector(image)

# Mostrar resultados por consola
for r in results:
    print(f"{r['label']}: {r['score']:.2f} ({r['box']})")

# Dibujar las detecciones sobre la imagen
draw = ImageDraw.Draw(image)
for r in results:
    box = r["box"]
    label = f"{r['label']} ({r['score']:.2f})"
    draw.rectangle(((box['xmin'], box['ymin']), (box['xmax'], box['ymax'])), outline="red", width=3)
    draw.text((box['xmin'], box['ymin'] - 10), label, fill="red")

# Mostrar la imagen con detecciones
plt.figure(figsize=(8,8))
plt.imshow(image)
plt.axis("off")
plt.show()

model.safetensors:   0%|          | 0.00/167M [00:00<?, ?B/s]